# **Organização dos dados**

### **Imports**

In [ ]:
import pandas as pd
import json
import os

### **Extração dos contextos e gabarito**

In [ ]:
def get_contexto_gabarito(arquivo, titulo):
    
    with open(arquivo, 'r', encoding='utf-8') as f:
        dados_json = json.load(f)

    texto_info = None
    for item in dados_json:
        if item.get("titulo") == titulo:
            texto_info = item
            break
    
    if not texto_info:
        return []

    texto_completo = texto_info['texto']
    respostas = texto_info['respostas']

    frases_finais = []
    indice_resposta = 0

    frases = texto_completo.replace('\n', ' ').split('.')

    for frase in frases:
        num_lacunas = frase.count('[LACUNA]')

        if not num_lacunas:
            continue

        partes = frase.split('[LACUNA]')
        
        respostas_locais = respostas[indice_resposta : indice_resposta + num_lacunas]

        for i in range(num_lacunas):
            preenchimentos = list(respostas_locais)
            preenchimentos[i] = '[LACUNA]'
            
            nova_frase = "".join(p + f for p, f in zip(partes, preenchimentos)) + partes[-1]
            
            frases_finais.append(nova_frase.strip())
        
        indice_resposta += num_lacunas

    return frases_finais, respostas

## **Mapeamento das correções e organização dos dados**

In [ ]:
MAP_CORRECAO = {
    'mesma classe e campo semântico diferente': 'classe_correta',
    'mesma classe e mesmo campo semântico': 'aceitavel',
    '-': 'incorreta'
}

dir_dados = '../data/raw'
dir_textos = '../data/textos.json'
dir_saida = '../data/processed/[human-eval]dados_teste-cloze.csv'

arquivos = os.listdir(dir_dados)

all_data = []

for arquivo in arquivos:
    caminho_completo_csv = os.path.join(dir_dados, arquivo)

    df_raw = pd.read_csv(caminho_completo_csv)

    titulo_texto = df_raw["texto"].iloc[0]
    
    contextos_texto, gabarito_texto = get_contexto_gabarito(dir_textos, titulo_texto)

    for _, row in df_raw.iterrows():
        for i in range(1, len(gabarito_texto)+1):
            lacuna_col = f'lacuna_{i}'
            class_col = f'classificacao_{i}'
            corr_col = f'correcao_{i}'
            
            if lacuna_col in row and pd.notna(row[lacuna_col]):
                
                correcao_original = row.get(corr_col, '-')
                correcao_mapeada = MAP_CORRECAO.get(correcao_original, 'incorreta')
                
                lacuna_aluno = str(row[lacuna_col]).strip().lower()
                resposta_correta = str(gabarito_texto[i-1]).strip().lower()
                
                if lacuna_aluno == resposta_correta:
                    correcao_mapeada = 'exata'

                all_data.append({
                    'contexto': contextos_texto[i-1],
                    'lacuna': row[lacuna_col],
                    'gabarito': gabarito_texto[i-1],
                    'classificacao': row.get(class_col, ""),
                    'correcao': correcao_mapeada
                })

In [ ]:
df_cloze = pd.DataFrame(all_data)

df_cloze.to_csv(dir_saida, index=False, encoding='utf-8')